In [1]:
%matplotlib qt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from copy import copy

from gauss import Gaussian
from Estimator_EKF import EKF
from Estimator_UKF import UKF
from Error_ellipse import e_ellipse

dt = 1
xt = np.array(np.zeros((3,11)))
zt = np.array(np.zeros((3,11)))

xt[:,0] = np.array([0,0,0])

u = np.array(np.zeros((2,10)))

wt = (2*np.pi)/20

R = np.array([[0.05,0,0],[0,0.05,0],[0,0,0.05]])
Q = np.array([[0.05,0,0],[0,0.05,0],[0,0,0.05]])

mov_noise_mean = np.array([0,0,0])
mes_noise_mean = np.array([0,0,0])

mov_noise = np.random.multivariate_normal(mov_noise_mean, R)        
xt[:,0] = xt[:,0] + mov_noise
    
mes_noise = np.random.multivariate_normal(mes_noise_mean, Q)
zt[:,0] = xt[:,0] + mes_noise

for i in range(1, len(xt[0])):
    u[0, i-1] = np.random.normal(2, 0.5)
    u[1, i-1] = np.random.normal(wt,0)
    
    x_prev = xt[0, i-1]
    y_prev = xt[1, i-1]
    theta_prev = xt[2, i-1]
    
    v = u[0,i-1]
    w = u[1,i-1]
   
    x_t = x_prev - (v/w)*np.sin(theta_prev) + (v/w)*np.sin(theta_prev + w*dt)
    y_t = y_prev + (v/w)*np.cos(theta_prev) - (v/w)*np.cos(theta_prev + w*dt)
    theta_t = theta_prev + w*dt
    
    mov_noise = np.random.multivariate_normal(mov_noise_mean, R)        
    xt[:,i] = np.array([x_t,y_t,theta_t])+mov_noise
    
    mes_noise = np.random.multivariate_normal(mes_noise_mean, Q)
    zt[:,i] = xt[:,i] + mes_noise
    #print(xt[:,i-1])


   

In [2]:
##UKF##
myUKF = UKF(R, Q)

estMeans = []
estSigs = []

estMeans.append(np.array([0,0,0]))
estSigs.append(np.array([[0.1, 0, 0],[0, 0.1, 0], [0, 0, 0.1]]))

for i in range (0, 1):
    #print(estimatedMeans[:, i])
    newMean, newSigma = myUKF.func_UKF(estMeans[i], estSigs[i], u[:, i], zt[:, i+1])
    estMeans.append(newMean)
    estSigs.append(newSigma)


estimatedLambdas = np.array(np.zeros((3,11)))
estimatedVs = np.array(np.zeros((3,3,11)))

fig0 = plt.figure(0)
ax0 = plt.subplot(111, aspect = 'equal')

e_ellipse.plot_eEllipse(estMeans, estSigs, ax0)

plt.scatter(xt[0,:], xt[1,:], c = "red", marker = "x")
plt.scatter(zt[0,:], zt[1,:], c = "blue", marker = ".")

i = 0
for x, y in zip(xt[0,:], xt[1,:]):
    plt.text(x, y, str(i), color="black", fontsize=12)
    i =i+1
plt.margins(0.1)
#    plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()


mean_predict [0. 0. 0.]
sigPointStar
 [-2.62886158 -0.41637077 -0.62831853]

mean_predict [-2.62886158 -0.41637077 -0.62831853]
sigPointStar
 [0.81532928 0.10409269 0.15707963]

mean_predict [-1.8135323  -0.31227808 -0.4712389 ]
sigPointStar
 [0.6572154  0.26220658 0.15707963]

mean_predict [-1.15631691 -0.0500715  -0.31415927]
sigPointStar
 [0.59225643 0.30331449 0.31519352]

mean_predict [-0.56406047  0.25324299  0.00103425]
sigPointStar
 [0.49910151 0.10409269 0.15707963]

mean_predict [-0.06495896  0.35733568  0.15811388]
sigPointStar
 [ 0.6572154  -0.05402119  0.15707963]

mean_predict [0.59225643 0.30331449 0.31519352]
sigPointStar
 [ 0.65699867 -0.10545192 -0.00103425]


In [3]:
##EKF##
myEKF = EKF(R, Q)

estMeans = []
estSigs = []

estMeans.append(np.array([0,0,0]))
estSigs.append(np.array([[0.1, 0, 0],[0, 0.1, 0], [0, 0, 0.1]]))


fig1 = plt.figure(1)
ax1 = plt.subplot(111, aspect = 'equal')

for i in range (0, len(u[0])):
    newMean, newSigma = myEKF.func_EKF(estMeans[i], estSigs[i], u[:, i], zt[:, i+1])
    estMeans.append(newMean)
    estSigs.append(newSigma)
e_ellipse.plot_eEllipse(estMeans, estSigs, ax1)


plt.scatter(xt[0,:], xt[1,:], c = "red", marker = "x")
plt.scatter(zt[0,:], zt[1,:], c = "blue", marker = ".")

i = 0
for x, y in zip(xt[0,:], xt[1,:]):
    plt.text(x, y, str(i), color="black", fontsize=12)
    i =i+1
plt.margins(0.1)
#    plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()


mean_predict
 [1.31443079 0.20818539 0.31415927]

mean_predict
 [1.98463098 0.13172642 0.48632398]

mean_predict
 [2.99769974 2.45395259 1.22556503]

mean_predict
 [3.7780138  4.44104871 1.38279925]

mean_predict
 [4.86024209 7.04636184 1.48481722]

mean_predict
 [5.32121823 8.47866228 1.71272869]

mean_predict
 [5.01224744 9.7584109  1.60654674]

mean_predict
 [ 5.28037594 11.55411697  1.65145576]

mean_predict
 [ 6.12967886 14.07483378  1.67098209]

mean_predict
 [ 5.14356495 14.97806914  2.20957   ]
